We first import pandas to read, parse, store and do anything to our dataframe followed by numpy for matrices and math functions. Levenshtein is used to get the closest string, used for typos.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1920px-Pandas_logo.svg.png" width="512" height="207">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/NumPy_logo_2020.svg/1920px-NumPy_logo_2020.svg.png" width="512" height="230">
<img src="https://ethw.org/w/images/8/83/Levenshtein.jpg" width="216" height="269">



In [115]:
import pandas as pd
import numpy as np
import Levenshtein as lev

We start by reading the "dataset.csv" file and make it a dataframe <br>
The delimiter in the dataset is ';'

In [116]:
csv = pd.read_csv("dataset.csv", sep=';')

Remove all comment columns since they're not 'data'

In [117]:
csv = csv.drop("Cancellation comments", axis=1)
csv = csv.drop("Departure delay comments", axis=1)
csv = csv.drop("Arrival delay comments", axis=1)

Remove all duplicates

In [118]:
csv = csv.drop_duplicates()

# Clean Date column
The date format must be %Y-%m (A year and a month) <br>
We replace all wrong delimiters by a '-' <br>
We convert all the strings to datetimes under the wanted format <br>
We exclude all data from before 2015 and after today <br>

In [119]:
csv["Date"] = csv["Date"].astype(str).str.replace(r"(\d{4})\w(\d{2})", r"\1-\2", regex=True)
csv["Date"] = pd.to_datetime(csv["Date"], errors="coerce", format="%Y-%m")
today = pd.to_datetime("today").normalize()
csv.loc[(csv.Date < "2015-01") | (csv.Date > today), "Date"] = pd.NaT

# Clean Service column
We convert the column to strings <br>
Using levenshtein's algorithm, we find the closest Service

In [120]:
csv["Service"] = csv["Service"].convert_dtypes(str)

services = ["NATIONAL", "INTERNATIONAL"]

max_val = 0
max_service = ""

for x in range(len(csv["Service"])):
    for service in services:
        if (x in csv["Service"]):
            a = lev.ratio(str(csv["Service"][x]).upper(), service)
            if (a > max_val and a > 0.7):
                max_val = a
                max_service = service
    max_val = 0
    csv["Service"][x] = max_service
    max_service = ""

# Clean Departure station
We convert the column to strings <br>
We remove any station name that contains numbers <br>
Using levenshtein's algorithm, we find the closest station

In [121]:
csv["Departure station"] = csv["Departure station"].convert_dtypes(str)
numbers = csv["Departure station"].str.contains(r".*\d.*", na=False)
csv.loc[numbers, "Departure station"] = np.nan
csv["Departure station"] = csv["Departure station"].str.upper()

cities_list = ['AIX EN PROVENCE TGV',
               'ANGERS SAINT LAUD',
               'ANGOULEME',
               'ANNECY',
               'ARRAS',
               'AVIGNON TGV',
               'BARCELONA',
               'BELLEGARDE (AIN)',
               'BESANCON FRANCHE COMTE TGV',
               'BORDEAUX ST JEAN',
               'BREST',
               'CHAMBERY CHALLES LES EAUX',
               'DIJON VILLE',
               'DOUAI', 
               'DUNKERQUE', 
               'FRANCFORT', 
               'GENEVE', 
               'GRENOBLE', 
               'ITALIE', 
               'LA ROCHELLE VILLE', 
               'LAUSANNE', 
               'LAVAL', 
               'LE CREUSOT MONTCEAU MONTCHANIN', 
               'LE MANS', 
               'LILLE', 
               'LYON PART DIEU', 
               'MACON LOCHE', 
               'MADRID', 
               'MARNE LA VALLEE', 
               'MARSEILLE ST CHARLES', 
               'METZ', 
               'MONTPELLIER', 
               'MULHOUSE VILLE', 
               'NANCY', 
               'NANTES', 
               'NICE VILLE', 
               'NIMES', 
               'PARIS EST', 
               'PARIS LYON', 
               'PARIS MONTPARNASSE', 
               'PARIS NORD', 
               'PARIS VAUGIRARD', 
               'PERPIGNAN', 
               'POTIIERS', 
               'QUIMPER', 
               'REIMS', 
               'RENNES', 
               'SAINT ETIENNE CHATEAUCREUX', 
               'ST MALO', 
               'ST PIERRE DES CORPS', 
               'STRASBOURG', 
               'STUTTGART', 
               'TOULON', 
               'TOULOUSE MATABIAU', 
               'TOURCOING', 
               'TOURS', 
               'VALENCE ALIXAN TGV', 
               'VANNES', 
               'ZURICH']

max_val = 0
max_city = ""

for x in range(len(csv["Departure station"])):
    for cities in cities_list:
        if (x in csv["Departure station"]):
            a = lev.ratio(csv["Departure station"][x], cities)
            if (a > max_val):
                max_val = a
                max_city = cities
    max_val = 0
    csv["Departure station"][x] = max_city
    max_city = ""

# Clean Arrival station
We convert the column to strings <br>
We remove any station name that contains numbers <br>
Using levenshtein's algorithm, we find the closest station

In [122]:
csv["Arrival station"] = csv["Arrival station"].convert_dtypes(str)
mask = csv["Arrival station"].str.contains(r".*\d.*", na=False)
csv.loc[mask, "Arrival station"] = np.nan
csv["Departure station"] = csv["Departure station"].str.upper()

max_val = 0
max_city = ""

for x in range(len(csv["Arrival station"])):
    for cities in cities_list:
        if (x in csv["Arrival station"]):
            a = lev.ratio(csv["Arrival station"][x], cities)
            if (a > max_val):
                max_val = a
                max_city = cities
    max_val = 0
    csv["Arrival station"][x] = max_city
    max_city = ""

# Clean Average journey time
We remove any data that contains letters <br>
We convert the column to floats <br>
We remove any journey time lower than 0 minutes (no time travellers here) <br>
We remove any journey time higher than 1500 minutes (no journey takes 25 hours right?) <br>

In [123]:
numbers_with_letters = csv["Average journey time"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Average journey time"] = np.nan
csv["Average journey time"] = csv["Average journey time"].convert_dtypes(float)
csv.loc[csv["Average journey time"] < 0, "Average journey time"] = np.nan
csv.loc[csv["Average journey time"] > 1500, "Average journey time"] = np.nan

# Clean Number of scheduled trains
We remove any data that contains letters <br>
We remove any float since an amount of trains can't be a float <br>
We remove any negative value (we don't want ghost trains) <br>
We convert the column to int

In [124]:
numbers_with_letters = csv["Number of scheduled trains"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of scheduled trains"] = np.nan
csv.loc[csv["Number of scheduled trains"] % 1 != 0, "Number of scheduled trains"] = np.nan
csv.loc[csv["Number of scheduled trains"] < 0, "Number of scheduled trains"] = np.nan
csv["Number of scheduled trains"] = csv["Number of scheduled trains"].convert_dtypes(int)

# Clean Number of cancelled trains
We remove any data that contains letters <br>
We remove any float since an amount of trains can't be a float <br>
We remove any negative value (we don't want ghost trains) <br>
We convert the column to int

In [125]:
numbers_with_letters = csv["Number of cancelled trains"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of cancelled trains"] = np.nan
csv.loc[csv["Number of cancelled trains"] % 1 != 0, "Number of cancelled trains"] = np.nan
csv.loc[csv["Number of scheduled trains"] < 0, "Number of scheduled trains"] = np.nan
csv["Number of cancelled trains"] = csv["Number of cancelled trains"].convert_dtypes(int)


Clean Number of trains delayed at departure

In [126]:
numbers_with_letters = csv["Number of trains delayed at departure"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of trains delayed at departure"] = np.nan
csv.loc[csv["Number of trains delayed at departure"] % 1 != 0, "Number of trains delayed at departure"] = np.nan
csv.loc[csv["Number of trains delayed at departure"] > csv["Number of scheduled trains"], "Number of trains delayed at departure"] = np.nan
csv["Number of trains delayed at departure"] = csv["Number of trains delayed at departure"].convert_dtypes(int)

Clean Average delay of late trains at departure

In [127]:
numbers_with_letters = csv["Average delay of late trains at departure"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Average delay of late trains at departure"] = np.nan
csv["Average delay of late trains at departure"] = csv["Average delay of late trains at departure"].convert_dtypes(float)
csv.loc[csv["Average delay of late trains at departure"] < 0, "Average delay of late trains at departure"] = np.nan
csv.loc[csv["Average delay of late trains at departure"] == 118.28872062060988, "Average delay of late trains at departure"] = np.nan

Clean Average delay of all trains at departure

In [128]:
numbers_with_letters = csv["Average delay of all trains at departure"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Average delay of all trains at departure"] = np.nan
csv["Average delay of all trains at departure"] = csv["Average delay of all trains at departure"].convert_dtypes(float)

Clean Number of trains delayed at arrival

In [129]:
numbers_with_letters = csv["Number of trains delayed at arrival"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of trains delayed at arrival"] = np.nan
csv.loc[csv["Number of trains delayed at arrival"] % 1 != 0, "Number of trains delayed at arrival"] = np.nan
csv.loc[csv["Number of trains delayed at arrival"] > csv["Number of scheduled trains"], "Number of trains delayed at arrival"] = np.nan
csv["Number of trains delayed at arrival"] = csv["Number of trains delayed at arrival"].convert_dtypes(int)

Clean Average delay of late trains at arrival

In [130]:
numbers_with_letters = csv["Average delay of late trains at arrival"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Average delay of late trains at arrival"] = np.nan
csv["Average delay of late trains at arrival"] = csv["Average delay of late trains at arrival"].convert_dtypes(float)
csv.loc[csv["Average delay of late trains at arrival"] < 0, "Average delay of late trains at arrival"] = np.nan
csv.loc[csv["Average delay of late trains at arrival"] > 330, "Average delay of late trains at arrival"] = np.nan


Clean Average delay of all trains at arrival

In [131]:
numbers_with_letters = csv["Average delay of all trains at arrival"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Average delay of all trains at arrival"] = np.nan
csv["Average delay of all trains at arrival"] = csv["Average delay of all trains at arrival"].convert_dtypes(float)


Clean Number of trains delayed > 15 min

In [132]:
numbers_with_letters = csv["Number of trains delayed > 15min"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of trains delayed > 15min"] = np.nan
csv.loc[csv["Number of trains delayed > 15min"] % 1 != 0, "Number of trains delayed > 15min"] = np.nan
csv.loc[csv["Number of trains delayed > 15min"] > csv["Number of trains delayed at departure"] + csv["Number of trains delayed at arrival"], "Number of trains delayed > 15min"] = np.nan
csv["Number of trains delayed > 15min"] = csv["Number of trains delayed > 15min"].convert_dtypes(int)

Clean Average delay of trains > 15min (if competing with flights)

In [133]:
numbers_with_letters = csv["Average delay of trains > 15min (if competing with flights)"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Average delay of trains > 15min (if competing with flights)"] = np.nan
csv["Average delay of trains > 15min (if competing with flights)"] = csv["Average delay of trains > 15min (if competing with flights)"].convert_dtypes(float)
csv.loc[csv["Average delay of trains > 15min (if competing with flights)"] > 330, "Average delay of trains > 15min (if competing with flights)"] = np.nan

Clean Number of trains delayed > 30min

In [134]:
numbers_with_letters = csv["Number of trains delayed > 30min"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of trains delayed > 30min"] = np.nan
csv.loc[csv["Number of trains delayed > 30min"] % 1 != 0, "Number of trains delayed > 30min"] = np.nan
csv.loc[csv["Number of trains delayed > 30min"] > csv["Number of trains delayed > 15min"], "Number of trains delayed > 30min"] = np.nan
csv["Number of trains delayed > 30min"] = csv["Number of trains delayed > 30min"].convert_dtypes(int)

Clean Number of trains delayed > 60min

In [135]:
numbers_with_letters = csv["Number of trains delayed > 60min"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Number of trains delayed > 60min"] = np.nan
csv.loc[csv["Number of trains delayed > 60min"] % 1 != 0, "Number of trains delayed > 60min"] = np.nan
csv.loc[csv["Number of trains delayed > 60min"] > csv["Number of trains delayed > 30min"], "Number of trains delayed > 60min"] = np.nan
csv["Number of trains delayed > 60min"] = csv["Number of trains delayed > 60min"].convert_dtypes(int)

Clean Pct delay due to external causes

In [136]:
numbers_with_letters = csv["Pct delay due to external causes"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Pct delay due to external causes"] = np.nan
csv["Pct delay due to external causes"] = csv["Pct delay due to external causes"].convert_dtypes(float)
csv.loc[(csv["Pct delay due to external causes"] > 100) | (csv["Pct delay due to external causes"] < 0), "Pct delay due to external causes"] = np.nan

Clean Pct delay due to infrastructure

In [137]:
numbers_with_letters = csv["Pct delay due to infrastructure"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Pct delay due to infrastructure"] = np.nan
csv["Pct delay due to infrastructure"] = csv["Pct delay due to infrastructure"].convert_dtypes(float)
csv.loc[(csv["Pct delay due to infrastructure"] > 100) | (csv["Pct delay due to infrastructure"] < 0), "Pct delay due to infrastructure"] = np.nan

Clean Pct delay due to traffic management

In [138]:
numbers_with_letters = csv["Pct delay due to traffic management"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Pct delay due to traffic management"] = np.nan
csv["Pct delay due to traffic management"] = csv["Pct delay due to traffic management"].convert_dtypes(float)
csv.loc[(csv["Pct delay due to traffic management"] > 99) | (csv["Pct delay due to traffic management"] < 0), "Pct delay due to traffic management"] = np.nan


Clean Pct delay due to rolling stock

In [139]:
numbers_with_letters = csv["Pct delay due to rolling stock"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Pct delay due to rolling stock"] = np.nan
csv["Pct delay due to rolling stock"] = csv["Pct delay due to rolling stock"].convert_dtypes(float)
csv.loc[(csv["Pct delay due to rolling stock"] > 99) | (csv["Pct delay due to rolling stock"] < 0), "Pct delay due to rolling stock"] = np.nan

Clean Pct delay due to station management and equipment reuse

In [140]:
numbers_with_letters = csv["Pct delay due to station management and equipment reuse"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Pct delay due to station management and equipment reuse"] = np.nan
csv["Pct delay due to station management and equipment reuse"] = csv["Pct delay due to station management and equipment reuse"].convert_dtypes(float)
csv.loc[(csv["Pct delay due to station management and equipment reuse"] > 100) | (csv["Pct delay due to station management and equipment reuse"] < 0), "Pct delay due to station management and equipment reuse"] = np.nan

Clean Pct delay due to passenger handling (crowding, disabled persons, connections)

In [141]:
numbers_with_letters = csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"].astype(str).str.contains(r"[a-zA-Z]", na=False)
csv.loc[numbers_with_letters, "Pct delay due to passenger handling (crowding, disabled persons, connections)"] = np.nan
csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"] = csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"].convert_dtypes(float)
csv.loc[(csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"] > 100) | (csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"] < 0), "Pct delay due to passenger handling (crowding, disabled persons, connections)"] = np.nan

# Final informations <br>
Out of 10659 final lines, 956 are perfect (no empty column), about 8.97% <br>

In [148]:
csv = csv.drop_duplicates()
csv_dropped = csv.dropna()

count = len(csv)
perfect_count = len(csv_dropped)

# print(f"Total lines: {count}", f"\nPerfect lines: {perfect_count}", f"\n~{round(perfect_count / count * 100, 2)}%")
csv = csv.sort_values(["Date", "Service", "Departure station", "Arrival station"])
csv.to_csv("cleaned_dataset.csv", index=False)

Total lines: 10659 
Perfect lines: 956 
~8.97%
